#Import necessary python packages to read data from Statsbomb Open Data Repo; then create two functions to read the season data for getting match_ids, and then read the match data 

In [1]:
import requests
import json
import pandas as pd
import tqdm
from pandas import json_normalize

def load_file(season_id, getter="remote", path = None):
    resp = requests.get(f"https://raw.githubusercontent.com/statsbomb/open-data/master/data/matches/43/{season_id}.json")
    season_dict = json.loads(resp.text)
    season_df = json_normalize(season_dict, sep="_")
    return season_dict, season_df

def load_match_file(match_id, getter="remote", path = None):
    resp = requests.get(f"https://raw.githubusercontent.com/statsbomb/open-data/master/data/events/{match_id}.json")
    match_dict = json.loads(resp.text)
    df = json_normalize(match_dict, sep="_")
    return match_dict, df

#WC2018 has season_id = '3'. USe that to extract the match_ids of the WC2018

In [2]:
season_dict, season_df = load_file('3', getter="remote")    
match_id_list = season_df.match_id.tolist() 

#Now collect all the games into a single dataframe

In [5]:
matches = []

for i in tqdm.tqdm(range(len(match_id_list)),desc='Loading all Matches'):
    match_dict, df = load_match_file(match_id_list[i], getter="remote")
    df['match_id'] = match_id_list[i]
    df['season_id'] = '3'
    matches.append(df)
masterdf = pd.concat(matches)

Loading all Matches: 100%|██████████| 64/64 [01:20<00:00,  1.26s/it]


#Save the dataframe as a parquet file (lightweight and faster to read than csv files)

In [6]:
masterdf.to_parquet('WC2018.parquet',index=None)